In [2]:
import Data_tools as dt
from config_paths import DATA, OUTPUT, INTERMEDIATE
import pandas as pd
import geopandas as gpd

### Raster resampling

In [ ]:
inpath = DATA/"External/Bathymetries/Europa/Po/Po_bathymetry_wsg84.tif"
outpath = DATA/"External/Bathymetries/Resampled_10/Po_resampled_10m.tif"

dt.resampling(inpath, outpath, 10)

### csv to parquet

In [6]:
in_csv = DATA/"Manual/Basins/csv/B_6226.csv"
out_parquet = OUTPUT/"Global_22_07_25/parquet/Only_swot_slope/slopes_swot_6226.parquet"

dt.csv_to_parquet(in_csv, out_parquet)

ConversionException: Conversion Error: CSV Error on Line: 512726
Original Line: "62266800030731",2025/01/05 11:31:39+00,-15.97010633,-
Error when converting column "lon". Could not convert string "-" to 'DOUBLE'

Column lon is being converted as type DOUBLE
This type was auto-detected from the CSV file.
Possible solutions:
* Override the type for this column manually by setting the type explicitly, e.g., types={'lon': 'VARCHAR'}
* Set the sample size to a larger value to enable the auto-detection to scan more values, e.g., sample_size=-1
* Use a COPY statement to automatically derive types from an existing table.
* Check whether the null string value is set correctly (e.g., nullstr = 'N/A')

  file = /Volumes/Science_SSD/Dissertation/0_data/Manual/Basins/csv/B_6226.csv
  delimiter = , (Auto-Detected)
  quote = " (Auto-Detected)
  escape = " (Auto-Detected)
  new_line = \n (Auto-Detected)
  header = true (Auto-Detected)
  skip_rows = 0 (Auto-Detected)
  comment = (empty) (Auto-Detected)

### Merging shapefiles living in a folder

In [ ]:
folder = DATA/"External/SWORD/shp_v16"
output_shapefile = DATA/"Manual/SWORD_v16_merged.shp"
dt.merge_shapefiles(folder, output_shapefile)

### Parquet to shapefile

In [ ]:

in_parquet = "/Users/daniel/Downloads/width_bc_b11.parquet"
out_shp = "/Users/daniel/Downloads/width_bc_b11.shp"
dt.parquet_to_shapefile(in_parquet, out_shp)

In [7]:
# Original parquet
df = pd.read_parquet(in_parquet)
print("Parquet rows:", len(df))

# Shapefile you wrote
gdf_shp = gpd.read_file(out_shp)
print("Shapefile rows:", len(gdf_shp))

# Compare a non-time field
print(df["width"].head(), gdf_shp["width"].head())


Parquet rows: 57153
Shapefile rows: 57153
0    83.870754
1    81.702714
2    88.666251
3    82.106470
4    92.736109
Name: width, dtype: float64 0    83.870754
1    81.702714
2    88.666251
3    82.106470
4    92.736109
Name: width, dtype: float64


### .parquet to csv

In [9]:
gdf = pd.read_parquet(OUTPUT/"Validation/Parquet/B77_width_bc_simple.parquet")
gdf.to_csv(OUTPUT/"Validation/csv/B77_width_bc_simple.csv", index=False)

In [8]:
node_to_drop = {77449300070471}
gdf = gdf[~gdf["node_id"].isin(node_to_drop)]
gdf.to_parquet(OUTPUT / "Validation/Parquet/B77_width_bc_simple.parquet", compression="zstd")

In [28]:
gdf['node_id'].nunique()

7726

In [5]:
erer = pd.read_csv(OUTPUT/"Global_22_07_25/simple/B77_huber_reg.csv")

In [6]:
erer['node_id'].nunique()

6615

In [7]:
len(gdf)

198200

In [30]:
len(erer)

176945